In [ ]:
# Importando as bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Carregamento dos Dados
df = pd.read_excel("conjunto-de-dados.xlsx")
# Separar descritores (X) e variável alvo (y)
X = df.drop(columns=["variavel-alvo"])
y = df["variavel-alvo"]
# Configuração do KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Para armazenar os resultados das métricas de cada divisão
mse_scores = []
r2_scores = []
mae_scores = []
histories = []

for train_index, test_index in kf.split(X):
    # Divisão em Treino e Teste
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Normalização dos Dados
    scaler_X = StandardScaler()
    X_train_scaled = scaler_X.fit_transform(X_train)
    X_test_scaled = scaler_X.transform(X_test)

    # Construção da Rede Neural
    model = Sequential()

    # Camada de entrada e primeira camada oculta
    model.add(Dense(128, input_dim=X_train_scaled.shape[1], activation='relu'))
    model.add(Dropout(0.2))

    # Segunda camada oculta
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))

    # Camada de saída
    model.add(Dense(1, activation='linear'))

    # Compilação do modelo
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mean_squared_error'])

    # Treinamento do Modelo
    history = model.fit(
        X_train_scaled, y_train,
        validation_split=0.2,
        epochs=100,
        batch_size=32,
        verbose=1
    )
    histories.append(history)

    # Avaliação do Modelo
    y_pred = model.predict(X_test_scaled)

    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)

    mse_scores.append(mse)
    r2_scores.append(r2)
    mae_scores.append(mae)

# Resultados Médios da Validação Cruzada
print(f"Validação Cruzada - Erro quadrático médio (MSE): {np.mean(mse_scores)} ± {np.std(mse_scores)}")
print(f"Validação Cruzada - R²: {np.mean(r2_scores)} ± {np.std(r2_scores)}")
print(f"Validação Cruzada - Erro absoluto médio (MAE): {np.mean(mae_scores)} ± {np.std(mae_scores)}")

# Visualização das Métricas de Treinamento
plt.figure(figsize=(12, 6))
plt.plot(histories[-1].history['loss'], label='Loss (Treino)')
plt.plot(histories[-1].history['val_loss'], label='Loss (Validação)')
plt.xlabel('Epochs')
plt.ylabel('Mean Squared Error')
plt.title('Evolução do Erro ao Longo das Épocas')
plt.legend()
plt.show()

# Visualização dos Resultados (Previsões vs Valores Reais)
y_test_last = y.iloc[test_index]
y_pred_last = model.predict(X_test_scaled)
plt.figure(figsize=(8, 6))
plt.scatter(y_test_last, y_pred_last, color='blue', label='Previsões')
plt.plot([min(y_test_last), max(y_test_last)], [min(y_test_last), max(y_test_last)], color='red', linestyle='--', label='Linha de identidade')
plt.xlabel('Valores Reais')
plt.ylabel('Previsões')
plt.title('Previsões vs. Valores Reais - Última Rodada')
plt.legend()
plt.show()